<h1>Example Retrieval System</h1>

In [1]:
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim import corpora, models, similarities
from operator import itemgetter

C:\Users\manue\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Sample corpus used as example

In [2]:
sample_corpus = [ "Human machine interface for lab abc computer applications",
                 "A survey of user opinion of computer system response time",
                 "The EPS user interface management system",
                 "System and human system engineering testing of EPS",
                 "Relation of user perceived response time to error measurement",
                 "The generation of random binary unordered trees",
                 "The intersection graph of paths in trees",
                 "Graph minors IV Widths of trees and well quasi ordering",
                 "Graph minors A survey" ]


In [3]:
def preprocess_document(doc):
    stopset = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    tokens = wordpunct_tokenize(doc)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and len(token) > 2]
    final = [stemmer.stem(word) for word in clean]
    return final

In [4]:
preprocess_document(sample_corpus[0])

['human', 'machin', 'interfac', 'lab', 'abc', 'comput', 'applic']

In [11]:
def create_dictionary(docs):
    pdocs = [preprocess_document(doc) for doc in docs]
    #print(pdocs)
    dictionary = corpora.Dictionary(pdocs)
    dictionary.save('vsm.dict')
    return dictionary

In [6]:
def get_keyword_to_id_mapping(dictionary):
    print(dictionary.token2id)

In [7]:
def docs2bows(corpus, dictionary):
    docs = [preprocess_document(d) for d in corpus]
    vectors = [dictionary.doc2bow(doc) for doc in docs]
    corpora.MmCorpus.serialize('vsm_docs.mm', vectors)
    return vectors

In [8]:
def create_TF_IDF_model(corpus):
    # should be already done previously
    dictionary = create_dictionary(corpus)
    docs2bows(corpus, dictionary) # already done
    
    # here comes the load of saved data
    loaded_corpus = corpora.MmCorpus('vsm_docs.mm')
    tfidf = models.TfidfModel(loaded_corpus)
    return tfidf, dictionary

In [16]:
def launch_query(corpus, q):
    # should be already done previously
    tfidf, dictionary = create_TF_IDF_model(corpus)
    loaded_corpus = corpora.MmCorpus('vsm_docs.mm')
    print(loaded_corpus[0])
    
    # convert query to vector space
    pq = preprocess_document(q)
    vq = dictionary.doc2bow(pq)
    
    qtfidf = tfidf[vq]
    
    # process the query
    index = similarities.MatrixSimilarity(loaded_corpus, num_features=len(dictionary))
    sim = index[qtfidf]
    
    ranking = sorted(enumerate(sim), key=itemgetter(1), reverse=True)
    for doc, score in ranking:
        print("[ Score = " + "%.3f" % round(score,3) + "] " + corpus[doc]); 

In [17]:
launch_query(sample_corpus, "Human machine interface")

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0)]
[ Score = 0.643] Human machine interface for lab abc computer applications
[ Score = 0.220] The EPS user interface management system
[ Score = 0.174] System and human system engineering testing of EPS
[ Score = 0.000] A survey of user opinion of computer system response time
[ Score = 0.000] Relation of user perceived response time to error measurement
[ Score = 0.000] The generation of random binary unordered trees
[ Score = 0.000] The intersection graph of paths in trees
[ Score = 0.000] Graph minors IV Widths of trees and well quasi ordering
[ Score = 0.000] Graph minors A survey
